In [5]:
%matplotlib inline

import itertools
from collections import Counter

import numpy as np
import pandas as pd

In [10]:
! wget --no-check-certificate https://raw.githubusercontent.com/mouna99/dien/master/data.tar.gz
! wget --no-check-certificate https://raw.githubusercontent.com/mouna99/dien/master/data1.tar.gz
! wget --no-check-certificate https://raw.githubusercontent.com/mouna99/dien/master/data2.tar.gz

--2019-09-07 17:45:10--  https://raw.githubusercontent.com/mouna99/dien/master/data.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.228.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81977637 (78M) [application/octet-stream]
Saving to: ‘data.tar.gz’

100%[======================================>] 81,977,637   105MB/s   in 0.7s   

2019-09-07 17:45:11 (105 MB/s) - ‘data.tar.gz’ saved [81977637/81977637]

--2019-09-07 17:45:11--  https://raw.githubusercontent.com/mouna99/dien/master/data1.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.228.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104430448 (100M) [application/octet-stream]
Saving to: ‘data1.tar.gz’

100%[====================================

In [15]:
! tar jxvf ./data.tar.gz && tar jxvf ./data1.tar.gz && tar jxvf ./data2.tar.gz

data/
data/cat_voc.pkl
data/mid_voc.pkl
data/uid_voc.pkl
data/local_train_splitByUser
data/local_test_splitByUser
data1/
data1/reviews-info
data2/
data2/item-info


In [16]:
TEST_RUN = False

In [17]:
train_df = pd.read_csv(
    "./data/local_train_splitByUser", sep='\t',
    names=['label', 'uid', 'mid', 'cat', 'hist_mids', 'hist_cats'])

valid_df = pd.read_csv(
    "./data/local_test_splitByUser", sep='\t',
    names=['label', 'uid', 'mid', 'cat', 'hist_mids', 'hist_cats'])

In [18]:
item_info_df = pd.read_csv("./data2/item-info", sep='\t', names=['mid', 'cat'])

In [19]:
item_info_df.head()

,mid,cat
0,0001048791,Books
1,0001048775,Books
2,0001048236,Books
3,0000401048,Books
4,0001019880,Books


In [20]:
reviews_info_df = pd.read_csv("./data1/reviews-info", sep='\t', names=['c1', 'mid', 'c3', 'c4'])

In [21]:
reviews_info_df.head()

,c1,mid,c3,c4
0,A10000012B7CGYKOMPQ4L,000100039X,5.0,1355616000
1,A2S166WSCFIFP5,000100039X,5.0,1071100800
2,A1BM81XB4QHOA3,000100039X,5.0,1390003200
3,A1MOSTXNIO5MPJ,000100039X,5.0,1317081600
4,A2XQ5LZHTD4AFT,000100039X,5.0,1033948800


In [22]:
reviews_info_df = reviews_info_df[['mid']].merge(item_info_df, on='mid', how='inner').drop_duplicates()

In [23]:
reviews_info_df.head()

,mid,cat
0,000100039X,Books
206,0001055178,Books
224,0001473123,Books
240,0001473727,Books
247,0001473905,Books


In [24]:
mid_cat_map = reviews_info_df.set_index('mid').to_dict()['cat']

In [25]:
if TEST_RUN:
    train_df = train_df.sample(1000)
    valid_df = valid_df.sample(1000)

In [26]:
# slow implement
def prepare_neg(df):
    records = df['hist_mids'].apply(lambda x: x.split(""))
    candidates = list(mid_cat_map.keys())
    max_len = len(candidates)

    def neg_sampling(filters, length):    
        mids = []
        cats = []
        for i in range(length):
            while(1):
                c = candidates[np.random.randint(0, max_len)]
                if c not in filters:
                    mids.append(c)
                    cats.append(mid_cat_map[c])
                    filters.add(c)
                    break
        return mids, cats
    
    total_neg_mids = []
    total_neg_cats = []
    for record in records:
        neg_mids, neg_cats = neg_sampling(set(record), len(record))
        total_neg_mids.append("".join(neg_mids))
        total_neg_cats.append("".join(neg_cats))
    
    return total_neg_mids, total_neg_cats

In [27]:
total_neg_mids, total_neg_cats = prepare_neg(train_df)
train_df['neg_hist_mids'] = total_neg_mids
train_df['neg_hist_cats'] = total_neg_cats

In [28]:
total_neg_mids, total_neg_cats = prepare_neg(valid_df)
valid_df['neg_hist_mids'] = total_neg_mids
valid_df['neg_hist_cats'] = total_neg_cats

In [29]:
train_df.to_csv('local_train.csv', sep='\t', index=False)
valid_df.to_csv('local_test.csv', sep='\t', index=False)